In [1]:
import gc
import multiprocessing
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import warnings
from scipy.stats import boxcox
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

In [2]:
df = pd.read_csv("../curated-data/ModelData_new.csv")
df.head(5)

,NASA,TA,EXT,AGR,CS,NT,OP,AV,EM,Task,...,BR,NP,SR,AP,AR,DWH,DWR,T,DS,Rank
0,24,38,2,8,10,3,8,16,17,26,...,2,2,1,1,4,2,70,1,1,2
1,17,35,6,9,7,4,8,25,14,31,...,2,2,1,1,5,1,70,1,1,1
2,20,38,8,8,6,5,7,19,22,29,...,1,1,1,2,4,1,90,2,2,0
3,16,54,3,7,3,9,6,23,24,16,...,1,2,0,2,2,2,60,1,2,0
4,18,28,6,4,10,3,8,13,12,35,...,1,1,1,2,5,1,60,2,1,2


In [3]:
df.SR.value_counts()

0    257
1    119
Name: SR, dtype: int64

In [4]:
df['CS'], lmbda = boxcox(df['CS'], lmbda=None)
df['OP'], lmbda = boxcox(df['OP'], lmbda=None)

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['SR']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

X_train = strat_train_set.drop('SR', axis=1)
Y_train = strat_train_set['SR'].copy()

X_test = strat_test_set.drop('SR', axis=1)
Y_test = strat_test_set['SR'].copy()

In [6]:
%%time
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 2000, 
                            min_samples_split = 10, 
                            min_samples_leaf = 2, 
                            max_features = 'auto', 
                            max_depth = None, 
                            bootstrap = True)
rf.fit(X_train, Y_train)

Wall time: 2.34 s


RandomForestClassifier(min_samples_leaf=2, min_samples_split=10,
                       n_estimators=2000)

In [7]:
y_pred = rf.predict(X_test)
y_score = rf.predict_proba(X_test)
pd.DataFrame(confusion_matrix(Y_test, y_pred))

,0,1
0,49,3
1,18,6


In [8]:
print("Accuracy: ", accuracy_score(Y_test, y_pred))
print("Precision: ", precision_score(Y_test, y_pred))
print("Recall: ", recall_score(Y_test, y_pred))
print("F1: ", f1_score(Y_test, y_pred))
print("Auc Score: ", roc_auc_score(Y_test, y_score[:,1]))

Accuracy:  0.7236842105263158
Precision:  0.6666666666666666
Recall:  0.25
F1:  0.36363636363636365
Auc Score:  0.6650641025641025
